In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint

Podaci se sastoje od 48x48 sivih slika lica.
Slike su unaprijed pridodjeljene jednoj od 7 mogućih kategorija (emocija) podijeljenjih na indexe od 0 do 6 - (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)

In [2]:
# Load the dataset and split it into training and validation sets
data = pd.read_csv("./archive/fer2013.csv")

train_data = data[data["Usage"] == "Training"]
test_data = data[data["Usage"] == "PrivateTest"]

x_train = train_data["pixels"].values
y_train = to_categorical(train_data["emotion"].values)

x_test = test_data["pixels"].values
y_test = to_categorical(test_data["emotion"].values)

In [3]:
# Define the CNN model
# dropout layer prevents overfitting
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(48, 48, 1), activation="relu"))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(7, activation="softmax"))

with open('static/model_summary.txt', 'w') as fh:
        model.summary(print_fn=lambda x: fh.write(x + '\n'))
model_json = model.to_json()
with open("static\model_arch.json", "w") as json_file:
    json_file.write(model_json)

In [4]:
# Preprocess the images
x_train = np.array([np.fromstring(x, dtype=int, sep=" ").reshape((48, 48, 1)) for x in x_train])
x_test = np.array([np.fromstring(x, dtype=int, sep=" ").reshape((48, 48, 1)) for x in x_test])
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [6]:
checkpointer = ModelCheckpoint('static/model_30.h5', save_best_only=True)
# Compile and fit the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), 
                    steps_per_epoch=len(x_train) / 32, 
                    epochs=30,
                    validation_data=datagen.flow(x_test, y_test, batch_size=32),
                    callbacks=[checkpointer]
            )

Epoch 1/30
897/897 [==============================] - 100s 111ms/step - loss: 0.6034 - accuracy: 0.7627 - val_loss: 1.6366 - val_accuracy: 0.5389
Epoch 2/30
897/897 [==============================] - 99s 110ms/step - loss: 0.5862 - accuracy: 0.7685 - val_loss: 1.7223 - val_accuracy: 0.5330
Epoch 3/30
897/897 [==============================] - 107s 119ms/step - loss: 0.5809 - accuracy: 0.7715 - val_loss: 1.6760 - val_accuracy: 0.5408
Epoch 4/30
897/897 [==============================] - 116s 130ms/step - loss: 0.5779 - accuracy: 0.7718 - val_loss: 1.6816 - val_accuracy: 0.5341
Epoch 5/30
897/897 [==============================] - 112s 125ms/step - loss: 0.5532 - accuracy: 0.7807 - val_loss: 1.6947 - val_accuracy: 0.5322
Epoch 6/30
897/897 [==============================] - 108s 120ms/step - loss: 0.5517 - accuracy: 0.7827 - val_loss: 1.6766 - val_accuracy: 0.5400
Epoch 7/30
897/897 [==============================] - 101s 113ms/step - loss: 0.5510 - accuracy: 0.7831 - val_loss: 1.6603 - 